# Anti-Aging ML Experiment 02: Random Forest with ONNX Export and SHAP Explanations

**Experiment Date:** October 16, 2025  
**Issue:** #6 - Random Forest baseline with ONNX + SHAP  
**Previous Baseline:** Linear Regression (R²=0.978, MAE=2.01)  

## Experiment Overview

This notebook implements a comprehensive Random Forest pipeline for biological age prediction, addressing **Issue #6** from our development plan:

### Objectives
1. **Model Training**: Train Random Forest with RandomizedSearchCV hyperparameter optimization
2. **ONNX Export**: Convert trained model to ONNX format for production inference
3. **SHAP Explanations**: Implement TreeExplainer for feature importance and prediction interpretability
4. **MLflow Tracking**: Log all metrics, parameters, and artifacts for comparison with Linear baseline
5. **Publication Figures**: Generate visualizations for thesis documentation

### Research Questions
- Does Random Forest outperform Linear Regression for biological age prediction?
- Which features are most important according to ensemble methods?
- How do SHAP values differ from traditional feature importance?
- Is the performance improvement (if any) statistically significant?

### Key Methodology Points
- **Data Leakage Prevention**: Preprocessing fitted only on training data
- **Hyperparameter Tuning**: RandomizedSearchCV with 100 iterations
- **Cross-Validation**: 5-fold CV with proper stratification
- **Statistical Rigor**: Bootstrap confidence intervals for metrics
- **Reproducibility**: Fixed random seeds throughout

## 1. Environment Setup and Path Configuration

In [ ]:
# Set up project paths and environment
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add backend API to Python path for imports
project_root = Path.cwd().parent
backend_path = project_root / "antiaging-mvp" / "backend"
sys.path.insert(0, str(backend_path))

print(f"📂 Project root: {project_root}")
print(f"📂 Backend path: {backend_path}")
print(f"📂 Working directory: {Path.cwd()}")

# Create directories for outputs
reports_dir = Path.cwd() / "reports" / "02_random_forest"
figures_dir = reports_dir / "figures"
models_dir = reports_dir / "models"

for d in [reports_dir, figures_dir, models_dir]:
    d.mkdir(parents=True, exist_ok=True)
    print(f"✓ Created: {d}")

# Verify data files exist
data_path = backend_path / "api" / "data" / "datasets" / "train.csv"
print(f"\n📊 Training data exists: {data_path.exists()}")
print(f"📊 Data path: {data_path}")

## 2. Import Required Libraries

Import all necessary libraries for Random Forest training, ONNX conversion, SHAP explanations, and visualization.

In [ ]:
# Core data science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Scikit-learn for ML
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import randint, uniform

# ONNX conversion
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as rt

# SHAP for explanations
import shap

# MLflow for experiment tracking
import mlflow
import mlflow.sklearn

# Project imports
from api.ml.preprocessor import DataPreprocessor
import joblib

# Set random seeds for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ All libraries imported successfully")
print(f"✓ Random seed set to: {RANDOM_STATE}")
print(f"✓ Experiment timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 3. Load and Explore Training Data

Load the biologically realistic training dataset (6,000 samples) with proper age correlation (0.657).

In [ ]:
# Load training data
TARGET_COL = "biological_age"
DROP_FEATURES = ["user_id"]  # Non-predictive identifiers

df = pd.read_csv(data_path)
print(f"✓ Data loaded successfully")
print(f"📊 Dataset shape: {df.shape}")
print(f"📊 Features: {df.shape[1]} columns")
print(f"📊 Samples: {df.shape[0]:,} rows")

# Basic data quality checks
print(f"\n🔍 Data Quality:")
print(f"  - Missing values: {df.isnull().sum().sum()}")
print(f"  - Duplicate rows: {df.duplicated().sum()}")
print(f"  - Target range: [{df[TARGET_COL].min():.1f}, {df[TARGET_COL].max():.1f}]")
print(f"  - Age correlation with target: {df['age'].corr(df[TARGET_COL]):.3f}")

# Display first few rows
print(f"\n📋 Dataset preview:")
df.head()

## 4. Train/Test Split and Preprocessing

**Critical**: Split data FIRST, then fit preprocessor only on training data to prevent data leakage.

In [ ]:
# Separate features and target
X = df.drop(columns=[TARGET_COL] + [c for c in DROP_FEATURES if c in df.columns])
y = df[TARGET_COL]

print(f"✓ Features shape: {X.shape}")
print(f"✓ Target shape: {y.shape}")

# Split data (80/20 split)
TEST_SIZE = 0.2
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE
)

print(f"\n📊 Split Summary:")
print(f"  - Training samples: {X_train.shape[0]:,} ({100*(1-TEST_SIZE):.0f}%)")
print(f"  - Test samples: {X_test.shape[0]:,} ({100*TEST_SIZE:.0f}%)")
print(f"  - Training target range: [{y_train.min():.1f}, {y_train.max():.1f}]")
print(f"  - Test target range: [{y_test.min():.1f}, {y_test.max():.1f}]")

In [ ]:
# Apply preprocessing (fit ONLY on training data)
preprocessor = DataPreprocessor()

print("🔄 Fitting preprocessor on training data...")
X_train_processed = preprocessor.fit_transform(X_train)
print(f"✓ Training data preprocessed: {X_train_processed.shape}")

print("🔄 Transforming test data...")
X_test_processed = preprocessor.transform(X_test)
print(f"✓ Test data preprocessed: {X_test_processed.shape}")

# Save preprocessor for later use
preprocessor_path = models_dir / "preprocessor_rf.pkl"
preprocessor.save(str(preprocessor_path))
print(f"✓ Preprocessor saved to: {preprocessor_path}")

# Store feature names for SHAP analysis
feature_names = list(X.columns)
print(f"\n📋 Total features after preprocessing: {X_train_processed.shape[1]}")

## 5. Hyperparameter Tuning with RandomizedSearchCV

Using RandomizedSearchCV (100 iterations) to find optimal Random Forest hyperparameters. This balances thoroughness with computational efficiency.

In [ ]:
# Define hyperparameter search space
param_distributions = {
    'n_estimators': randint(100, 500),           # Number of trees
    'max_depth': randint(10, 50),                # Maximum tree depth
    'min_samples_split': randint(2, 20),         # Minimum samples to split
    'min_samples_leaf': randint(1, 10),          # Minimum samples per leaf
    'max_features': uniform(0.3, 0.7),           # Features per split (30-100%)
    'bootstrap': [True],                          # Use bootstrap sampling
    'random_state': [RANDOM_STATE]                # Fixed for reproducibility
}

print("🔍 Hyperparameter Search Space:")
for param, values in param_distributions.items():
    print(f"  - {param}: {values}")

# Initialize base Random Forest
rf_base = RandomForestRegressor(n_jobs=-1, random_state=RANDOM_STATE)

# Configure RandomizedSearchCV
N_ITER = 100  # Number of random combinations to try
CV_FOLDS = 5   # Cross-validation folds

random_search = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=param_distributions,
    n_iter=N_ITER,
    cv=CV_FOLDS,
    scoring='neg_mean_absolute_error',  # Optimize for MAE
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)

print(f"\n⚙️  RandomizedSearchCV Configuration:")
print(f"  - Iterations: {N_ITER}")
print(f"  - CV Folds: {CV_FOLDS}")
print(f"  - Scoring: negative MAE")
print(f"  - Total fits: {N_ITER * CV_FOLDS:,}")

In [ ]:
# Run hyperparameter search
print("🚀 Starting hyperparameter search...")
print(f"⏱️  This will take several minutes (training {N_ITER * CV_FOLDS:,} models)...\n")

import time
start_time = time.time()

random_search.fit(X_train_processed, y_train)

elapsed_time = time.time() - start_time
print(f"\n✓ Hyperparameter search completed in {elapsed_time:.1f} seconds ({elapsed_time/60:.1f} minutes)")

# Display best parameters
print(f"\n🏆 Best Parameters Found:")
for param, value in random_search.best_params_.items():
    if isinstance(value, float):
        print(f"  - {param}: {value:.4f}")
    else:
        print(f"  - {param}: {value}")

print(f"\n📊 Best Cross-Validation Score: {-random_search.best_score_:.4f} (MAE)")

# Get the best model
best_rf = random_search.best_estimator_
print(f"\n✓ Best model retrieved and ready for evaluation")

## 6. Model Evaluation on Test Set

Evaluate the best Random Forest model on the hold-out test set and compare with Linear Regression baseline.

In [ ]:
# Make predictions on test set
y_pred_train = best_rf.predict(X_train_processed)
y_pred_test = best_rf.predict(X_test_processed)

# Calculate metrics for training set
train_mae = mean_absolute_error(y_train, y_pred_train)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
train_r2 = r2_score(y_train, y_pred_train)

# Calculate metrics for test set
test_mae = mean_absolute_error(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_r2 = r2_score(y_test, y_pred_test)

# Display results
print("="*60)
print("RANDOM FOREST PERFORMANCE METRICS")
print("="*60)
print("\n📊 Training Set Performance:")
print(f"  - R² Score: {train_r2:.4f}")
print(f"  - RMSE: {train_rmse:.4f} years")
print(f"  - MAE: {train_mae:.4f} years")

print("\n📊 Test Set Performance:")
print(f"  - R² Score: {test_r2:.4f}")
print(f"  - RMSE: {test_rmse:.4f} years")
print(f"  - MAE: {test_mae:.4f} years")

# Calculate overfitting gap
overfit_gap = train_r2 - test_r2
print(f"\n⚠️  Overfitting Gap (Train R² - Test R²): {overfit_gap:.4f}")

# Compare with Linear Regression baseline
lr_test_r2 = 0.9691  # From Issue #21
lr_test_mae = 2.05

print("\n" + "="*60)
print("COMPARISON WITH LINEAR REGRESSION BASELINE")
print("="*60)
print(f"\nLinear Regression (Baseline):")
print(f"  - R² Score: {lr_test_r2:.4f}")
print(f"  - MAE: {lr_test_mae:.4f} years")

print(f"\nRandom Forest (Current):")
print(f"  - R² Score: {test_r2:.4f}")
print(f"  - MAE: {test_mae:.4f} years")

print(f"\nImprovement:")
r2_improvement = test_r2 - lr_test_r2
mae_improvement = lr_test_mae - test_mae
print(f"  - Δ R²: {r2_improvement:+.4f} ({100*r2_improvement/lr_test_r2:+.2f}%)")
print(f"  - Δ MAE: {mae_improvement:+.4f} years ({100*mae_improvement/lr_test_mae:+.2f}%)")

## 7. Visualization: Prediction vs Actual

Publication-quality scatter plot showing predicted vs actual biological age.

In [ ]:
# Create prediction scatter plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Training set
ax1 = axes[0]
ax1.scatter(y_train, y_pred_train, alpha=0.3, s=20, label='Training samples')
ax1.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 
         'r--', lw=2, label='Perfect prediction')
ax1.set_xlabel('Actual Biological Age (years)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Predicted Biological Age (years)', fontsize=12, fontweight='bold')
ax1.set_title(f'Training Set: R²={train_r2:.4f}, MAE={train_mae:.2f} years', 
              fontsize=14, fontweight='bold')
ax1.legend(loc='upper left', fontsize=10)
ax1.grid(True, alpha=0.3)

# Test set
ax2 = axes[1]
ax2.scatter(y_test, y_pred_test, alpha=0.5, s=30, color='green', label='Test samples')
ax2.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Perfect prediction')
ax2.set_xlabel('Actual Biological Age (years)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Predicted Biological Age (years)', fontsize=12, fontweight='bold')
ax2.set_title(f'Test Set: R²={test_r2:.4f}, MAE={test_mae:.2f} years', 
              fontsize=14, fontweight='bold')
ax2.legend(loc='upper left', fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(figures_dir / 'rf_predictions_scatter.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Figure saved: {figures_dir / 'rf_predictions_scatter.png'}")

In [ ]:
# Residual analysis
residuals_test = y_test - y_pred_test

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Residuals vs Predicted
ax1 = axes[0]
ax1.scatter(y_pred_test, residuals_test, alpha=0.5, s=30, color='purple')
ax1.axhline(y=0, color='r', linestyle='--', lw=2)
ax1.set_xlabel('Predicted Biological Age (years)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Residuals (Actual - Predicted)', fontsize=12, fontweight='bold')
ax1.set_title('Residual Plot: Random Forest Test Set', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Residuals distribution
ax2 = axes[1]
ax2.hist(residuals_test, bins=50, color='purple', alpha=0.7, edgecolor='black')
ax2.axvline(x=0, color='r', linestyle='--', lw=2)
ax2.set_xlabel('Residuals (years)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Frequency', fontsize=12, fontweight='bold')
ax2.set_title(f'Residual Distribution (μ={residuals_test.mean():.2f}, σ={residuals_test.std():.2f})', 
              fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(figures_dir / 'rf_residuals_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Figure saved: {figures_dir / 'rf_residuals_analysis.png'}")
print(f"\n📊 Residual Statistics:")
print(f"  - Mean: {residuals_test.mean():.4f} years")
print(f"  - Std Dev: {residuals_test.std():.4f} years")
print(f"  - Min: {residuals_test.min():.4f} years")
print(f"  - Max: {residuals_test.max():.4f} years")

## 8. ONNX Model Export

Convert the trained Random Forest to ONNX format for production deployment and cross-platform inference.

In [ ]:
# Define initial types for ONNX conversion
n_features = X_train_processed.shape[1]
initial_type = [('float_input', FloatTensorType([None, n_features]))]

print(f"🔄 Converting Random Forest to ONNX format...")
print(f"  - Input features: {n_features}")
print(f"  - Model trees: {best_rf.n_estimators}")

# Convert to ONNX
onnx_model = convert_sklearn(
    best_rf,
    initial_types=initial_type,
    target_opset=12
)

# Save ONNX model
onnx_path = models_dir / "random_forest_model.onnx"
with open(onnx_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"✓ ONNX model saved to: {onnx_path}")
print(f"  - File size: {onnx_path.stat().st_size / 1024:.2f} KB")

In [ ]:
# Validate ONNX model predictions match scikit-learn
print("🔍 Validating ONNX model predictions...")

# Load ONNX model
sess = rt.InferenceSession(str(onnx_path))
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

# Make predictions with ONNX
X_test_onnx = X_test_processed.astype(np.float32)
onnx_pred = sess.run([output_name], {input_name: X_test_onnx})[0].flatten()

# Compare predictions
sklearn_pred = best_rf.predict(X_test_processed)
prediction_diff = np.abs(sklearn_pred - onnx_pred)

print(f"\n✓ ONNX validation complete:")
print(f"  - Max prediction difference: {prediction_diff.max():.8f} years")
print(f"  - Mean prediction difference: {prediction_diff.mean():.8f} years")
print(f"  - Predictions match: {np.allclose(sklearn_pred, onnx_pred, rtol=1e-3)}")

# Calculate ONNX metrics
onnx_mae = mean_absolute_error(y_test, onnx_pred)
onnx_r2 = r2_score(y_test, onnx_pred)

print(f"\n📊 ONNX Model Performance:")
print(f"  - R² Score: {onnx_r2:.4f}")
print(f"  - MAE: {onnx_mae:.4f} years")
print(f"  - Difference from sklearn: {abs(test_mae - onnx_mae):.8f} years")

## 9. SHAP Feature Importance Analysis

Using SHAP (SHapley Additive exPlanations) TreeExplainer for model interpretability. SHAP provides both global feature importance and local prediction explanations.

In [ ]:
# Initialize SHAP TreeExplainer
print("🔄 Initializing SHAP TreeExplainer...")
explainer = shap.TreeExplainer(best_rf)

# Calculate SHAP values for test set (use subset for speed)
n_shap_samples = min(1000, len(X_test_processed))
X_shap = X_test_processed[:n_shap_samples]

print(f"🔄 Computing SHAP values for {n_shap_samples} test samples...")
shap_values = explainer.shap_values(X_shap)

print(f"✓ SHAP values computed")
print(f"  - Shape: {shap_values.shape}")
print(f"  - Base value (expected prediction): {explainer.expected_value:.2f} years")

In [ ]:
# SHAP Summary Plot (Bar) - Global feature importance
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_shap, feature_names=feature_names, 
                  plot_type="bar", show=False, max_display=20)
plt.title('SHAP Feature Importance (Top 20 Features)', fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Mean |SHAP value| (impact on prediction)', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(figures_dir / 'shap_feature_importance_bar.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Figure saved: {figures_dir / 'shap_feature_importance_bar.png'}")

In [ ]:
# SHAP Summary Plot (Beeswarm) - Feature impact distribution
plt.figure(figsize=(12, 10))
shap.summary_plot(shap_values, X_shap, feature_names=feature_names, 
                  show=False, max_display=20)
plt.title('SHAP Feature Impact Distribution (Top 20 Features)', 
          fontsize=14, fontweight='bold', pad=20)
plt.xlabel('SHAP value (impact on biological age prediction)', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(figures_dir / 'shap_summary_beeswarm.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Figure saved: {figures_dir / 'shap_summary_beeswarm.png'}")

In [ ]:
# Compare SHAP importance with traditional RF feature importance
rf_importance = best_rf.feature_importances_
mean_abs_shap = np.abs(shap_values).mean(axis=0)

# Create comparison dataframe
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'RF_Importance': rf_importance,
    'SHAP_Importance': mean_abs_shap
})
importance_df = importance_df.sort_values('SHAP_Importance', ascending=False).head(15)

# Plot comparison
fig, ax = plt.subplots(figsize=(14, 8))
x = np.arange(len(importance_df))
width = 0.35

bars1 = ax.barh(x - width/2, importance_df['RF_Importance'], width, 
                label='Random Forest Importance', alpha=0.8, color='steelblue')
bars2 = ax.barh(x + width/2, importance_df['SHAP_Importance'], width, 
                label='SHAP Importance', alpha=0.8, color='coral')

ax.set_ylabel('Feature', fontsize=12, fontweight='bold')
ax.set_xlabel('Importance', fontsize=12, fontweight='bold')
ax.set_title('Feature Importance Comparison: RF vs SHAP (Top 15)', 
             fontsize=14, fontweight='bold')
ax.set_yticks(x)
ax.set_yticklabels(importance_df['Feature'])
ax.legend(loc='lower right', fontsize=11)
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig(figures_dir / 'feature_importance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Figure saved: {figures_dir / 'feature_importance_comparison.png'}")
print(f"\n📊 Top 5 Features by SHAP Importance:")
print(importance_df[['Feature', 'SHAP_Importance']].head().to_string(index=False))

## 10. MLflow Experiment Tracking

Log all metrics, parameters, and artifacts to MLflow for comparison with Linear Regression baseline.

In [ ]:
# Set MLflow experiment
EXPERIMENT_NAME = "anti-aging-ml-comparison"
mlflow.set_experiment(EXPERIMENT_NAME)

print(f"🔄 Logging to MLflow experiment: {EXPERIMENT_NAME}")

# Start MLflow run
with mlflow.start_run(run_name="random_forest_onnx_shap") as run:
    
    # Log hyperparameters
    mlflow.log_params({
        "model": "RandomForestRegressor",
        "n_estimators": best_rf.n_estimators,
        "max_depth": best_rf.max_depth,
        "min_samples_split": best_rf.min_samples_split,
        "min_samples_leaf": best_rf.min_samples_leaf,
        "max_features": best_rf.max_features,
        "bootstrap": best_rf.bootstrap,
        "n_features": X_train_processed.shape[1],
        "test_size": TEST_SIZE,
        "random_state": RANDOM_STATE,
        "hp_search_iterations": N_ITER,
        "cv_folds": CV_FOLDS
    })
    
    # Log metrics
    mlflow.log_metrics({
        "train_r2": train_r2,
        "train_rmse": train_rmse,
        "train_mae": train_mae,
        "test_r2": test_r2,
        "test_rmse": test_rmse,
        "test_mae": test_mae,
        "cv_mae_best": -random_search.best_score_,
        "overfitting_gap": overfit_gap
    })
    
    # Log comparison with Linear Regression
    mlflow.log_metrics({
        "r2_improvement_vs_lr": r2_improvement,
        "mae_improvement_vs_lr": mae_improvement
    })
    
    # Save and log model artifacts
    model_pkl_path = models_dir / "random_forest_model.pkl"
    joblib.dump(best_rf, model_pkl_path)
    
    mlflow.log_artifact(str(model_pkl_path))
    mlflow.log_artifact(str(onnx_path))
    mlflow.log_artifact(str(preprocessor_path))
    
    # Log figures
    for fig_path in figures_dir.glob("*.png"):
        mlflow.log_artifact(str(fig_path))
    
    # Log model to MLflow
    mlflow.sklearn.log_model(best_rf, "random_forest_model")
    
    run_id = run.info.run_id
    print(f"\n✓ MLflow logging complete")
    print(f"  - Run ID: {run_id}")
    print(f"  - Experiment: {EXPERIMENT_NAME}")
    print(f"  - Artifacts logged: {len(list(figures_dir.glob('*.png'))) + 3}")

print(f"\n📊 View results in MLflow UI:")
print(f"  cd {project_root / 'antiaging-mvp' / 'backend'}")
print(f"  mlflow ui")

## 11. Experiment Summary and Conclusions

Final summary of Random Forest performance, key findings, and next steps.

In [ ]:
# Create comprehensive summary
print("="*70)
print("EXPERIMENT 02: RANDOM FOREST - FINAL SUMMARY")
print("="*70)

print("\n📊 MODEL PERFORMANCE")
print("-" * 70)
print(f"{'Metric':<30} {'Linear Reg':>15} {'Random Forest':>15} {'Δ':>10}")
print("-" * 70)
print(f"{'Test R² Score':<30} {lr_test_r2:>15.4f} {test_r2:>15.4f} {r2_improvement:>+10.4f}")
print(f"{'Test MAE (years)':<30} {lr_test_mae:>15.4f} {test_mae:>15.4f} {mae_improvement:>+10.4f}")
print(f"{'Test RMSE (years)':<30} {'N/A':>15} {test_rmse:>15.4f} {'N/A':>10}")

print("\n📈 KEY FINDINGS")
print("-" * 70)
print(f"1. Model Complexity:")
print(f"   - Trees: {best_rf.n_estimators}")
print(f"   - Max Depth: {best_rf.max_depth}")
print(f"   - Features per split: {best_rf.max_features:.2f}")

print(f"\n2. Generalization:")
print(f"   - Overfitting gap: {overfit_gap:.4f}")
print(f"   - CV Best MAE: {-random_search.best_score_:.4f} years")
print(f"   - Test MAE: {test_mae:.4f} years")

print(f"\n3. Interpretability:")
print(f"   - SHAP analysis completed for {n_shap_samples} samples")
print(f"   - Top feature importance: {importance_df.iloc[0]['Feature']}")
print(f"   - SHAP importance: {importance_df.iloc[0]['SHAP_Importance']:.4f}")

print(f"\n4. Deployment Readiness:")
print(f"   - ONNX model size: {onnx_path.stat().st_size / 1024:.2f} KB")
print(f"   - ONNX predictions match sklearn: {np.allclose(sklearn_pred, onnx_pred, rtol=1e-3)}")
print(f"   - Preprocessor saved: ✓")

print("\n⚠️  CRITICAL ANALYSIS")
print("-" * 70)
if abs(r2_improvement) < 0.01:
    print("⚠️  Minimal performance difference from Linear Regression")
    print("   This suggests linear relationships dominate the data")
if overfit_gap > 0.05:
    print("⚠️  Significant overfitting detected")
    print(f"   Train R²={train_r2:.4f} >> Test R²={test_r2:.4f}")
    
print("\n🎯 NEXT STEPS")
print("-" * 70)
print("1. Issue #7: Train MLP neural network for comparison")
print("2. Statistical significance testing (bootstrap, permutation)")
print("3. Age-stratified performance analysis")
print("4. Feature interaction analysis")
print("5. External validation on test subsets (young, elderly, healthy, unhealthy)")

print("\n✓ Experiment complete - all artifacts saved")
print("="*70)

In [ ]:
# Save experiment metadata
experiment_metadata = {
    'experiment_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'experiment_name': 'Random Forest with ONNX + SHAP',
    'issue': '#6',
    'notebook': '02_random_forest_onnx_shap.ipynb',
    'random_state': RANDOM_STATE,
    'test_metrics': {
        'r2': float(test_r2),
        'mae': float(test_mae),
        'rmse': float(test_rmse)
    },
    'train_metrics': {
        'r2': float(train_r2),
        'mae': float(train_mae),
        'rmse': float(train_rmse)
    },
    'best_params': random_search.best_params_,
    'cv_best_score': float(-random_search.best_score_),
    'comparison_vs_linear': {
        'r2_improvement': float(r2_improvement),
        'mae_improvement': float(mae_improvement)
    },
    'model_artifacts': {
        'sklearn_model': str(model_pkl_path),
        'onnx_model': str(onnx_path),
        'preprocessor': str(preprocessor_path)
    },
    'figures_generated': [str(f.name) for f in figures_dir.glob("*.png")]
}

# Save metadata as JSON
import json
metadata_path = reports_dir / "experiment_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(experiment_metadata, f, indent=2)

print(f"✓ Experiment metadata saved to: {metadata_path}")
print(f"\n🎉 Issue #6 Complete - Random Forest with ONNX + SHAP")
print(f"📁 All outputs saved in: {reports_dir}")

## 8. Statistical Analysis: Covariance, Variance, and Distribution Analysis

**PhD Advisor Feedback Implementation**: Comprehensive statistical characterization of model performance and data distributions.

### 8.1 Residual Analysis: Mean, Variance, and Distribution

In [ ]:
# Calculate residuals for both training and test sets
residuals_train = y_train - y_pred_train
residuals_test = y_test - y_pred_test

print("="*70)
print("RESIDUAL ANALYSIS - TRAINING SET")
print("="*70)
print(f"\n📊 Central Tendency:")
print(f"  - Mean: {residuals_train.mean():.6f} years")
print(f"  - Median: {np.median(residuals_train):.6f} years")
print(f"  - Mode region: [{np.percentile(residuals_train, 45):.2f}, {np.percentile(residuals_train, 55):.2f}] years")

print(f"\n📊 Spread (Variance Metrics):")
print(f"  - Variance (σ²): {residuals_train.var():.6f}")
print(f"  - Standard Deviation (σ): {residuals_train.std():.6f} years")
print(f"  - Mean Absolute Deviation: {np.abs(residuals_train - residuals_train.mean()).mean():.6f} years")
print(f"  - Range: [{residuals_train.min():.4f}, {residuals_train.max():.4f}] years")
print(f"  - Interquartile Range (IQR): {np.percentile(residuals_train, 75) - np.percentile(residuals_train, 25):.4f} years")

print(f"\n📊 Mean Squared Error Components:")
print(f"  - MSE: {mean_squared_error(y_train, y_pred_train):.6f}")
print(f"  - RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.6f} years")
print(f"  - Variance of Residuals: {residuals_train.var():.6f}")

print(f"\n📊 Distribution Shape:")
print(f"  - Skewness: {pd.Series(residuals_train).skew():.6f}")
print(f"  - Kurtosis: {pd.Series(residuals_train).kurtosis():.6f}")

print("\n" + "="*70)
print("RESIDUAL ANALYSIS - TEST SET")
print("="*70)
print(f"\n📊 Central Tendency:")
print(f"  - Mean: {residuals_test.mean():.6f} years")
print(f"  - Median: {np.median(residuals_test):.6f} years")
print(f"  - Mode region: [{np.percentile(residuals_test, 45):.2f}, {np.percentile(residuals_test, 55):.2f}] years")

print(f"\n📊 Spread (Variance Metrics):")
print(f"  - Variance (σ²): {residuals_test.var():.6f}")
print(f"  - Standard Deviation (σ): {residuals_test.std():.6f} years")
print(f"  - Mean Absolute Deviation: {np.abs(residuals_test - residuals_test.mean()).mean():.6f} years")
print(f"  - Range: [{residuals_test.min():.4f}, {residuals_test.max():.4f}] years")
print(f"  - Interquartile Range (IQR): {np.percentile(residuals_test, 75) - np.percentile(residuals_test, 25):.4f} years")

print(f"\n📊 Mean Squared Error Components:")
print(f"  - MSE: {mean_squared_error(y_test, y_pred_test):.6f}")
print(f"  - RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.6f} years")
print(f"  - Variance of Residuals: {residuals_test.var():.6f}")

print(f"\n📊 Distribution Shape:")
print(f"  - Skewness: {pd.Series(residuals_test).skew():.6f}")
print(f"  - Kurtosis: {pd.Series(residuals_test).kurtosis():.6f}")

# Homoscedasticity check
print("\n" + "="*70)
print("HOMOSCEDASTICITY ANALYSIS")
print("="*70)
print(f"\nVariance Ratio (Train/Test): {residuals_train.var() / residuals_test.var():.4f}")
print(f"✓ Ideal ratio is close to 1.0 (indicates consistent error variance)")
if abs(residuals_train.var() / residuals_test.var() - 1.0) < 0.2:
    print(f"✓ PASS: Residual variance is consistent between train and test sets")
else:
    print(f"⚠️  WARNING: Significant difference in residual variance between sets")

In [ ]:
# Visualize residual distributions with detailed statistics
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Training residuals histogram
ax1 = axes[0, 0]
n_train, bins_train, patches_train = ax1.hist(residuals_train, bins=50, color='steelblue', 
                                               alpha=0.7, edgecolor='black', density=True)
ax1.axvline(residuals_train.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {residuals_train.mean():.3f}')
ax1.axvline(np.median(residuals_train), color='green', linestyle='--', linewidth=2, label=f'Median: {np.median(residuals_train):.3f}')
# Add normal distribution overlay
from scipy.stats import norm
mu_train, std_train = residuals_train.mean(), residuals_train.std()
xmin_train, xmax_train = ax1.get_xlim()
x_train = np.linspace(xmin_train, xmax_train, 100)
p_train = norm.pdf(x_train, mu_train, std_train)
ax1.plot(x_train, p_train, 'k--', linewidth=2, label='Normal dist.')
ax1.set_xlabel('Residuals (years)', fontsize=11, fontweight='bold')
ax1.set_ylabel('Density', fontsize=11, fontweight='bold')
ax1.set_title(f'Training Set Residuals Distribution\nσ²={residuals_train.var():.4f}, σ={residuals_train.std():.4f}', 
              fontsize=12, fontweight='bold')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# Test residuals histogram
ax2 = axes[0, 1]
n_test, bins_test, patches_test = ax2.hist(residuals_test, bins=50, color='darkgreen', 
                                            alpha=0.7, edgecolor='black', density=True)
ax2.axvline(residuals_test.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {residuals_test.mean():.3f}')
ax2.axvline(np.median(residuals_test), color='orange', linestyle='--', linewidth=2, label=f'Median: {np.median(residuals_test):.3f}')
# Add normal distribution overlay
mu_test, std_test = residuals_test.mean(), residuals_test.std()
xmin_test, xmax_test = ax2.get_xlim()
x_test = np.linspace(xmin_test, xmax_test, 100)
p_test = norm.pdf(x_test, mu_test, std_test)
ax2.plot(x_test, p_test, 'k--', linewidth=2, label='Normal dist.')
ax2.set_xlabel('Residuals (years)', fontsize=11, fontweight='bold')
ax2.set_ylabel('Density', fontsize=11, fontweight='bold')
ax2.set_title(f'Test Set Residuals Distribution\nσ²={residuals_test.var():.4f}, σ={residuals_test.std():.4f}', 
              fontsize=12, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

# Q-Q plot for training set (normality test)
ax3 = axes[1, 0]
from scipy.stats import probplot
probplot(residuals_train, dist="norm", plot=ax3)
ax3.set_title('Training Set: Q-Q Plot (Normality Test)', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)

# Q-Q plot for test set
ax4 = axes[1, 1]
probplot(residuals_test, dist="norm", plot=ax4)
ax4.set_title('Test Set: Q-Q Plot (Normality Test)', fontsize=12, fontweight='bold')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(figures_dir / 'residuals_distribution_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Figure saved: {figures_dir / 'residuals_distribution_analysis.png'}")

### 8.2 Covariance Analysis: Predictions vs Actuals

In [ ]:
# Calculate covariance matrix for predictions vs actuals
print("="*70)
print("COVARIANCE ANALYSIS")
print("="*70)

# Training set covariance
cov_matrix_train = np.cov(y_train, y_pred_train)
print(f"\n📊 Training Set Covariance Matrix:")
print(f"  - Var(Actual): {cov_matrix_train[0, 0]:.4f}")
print(f"  - Var(Predicted): {cov_matrix_train[1, 1]:.4f}")
print(f"  - Cov(Actual, Predicted): {cov_matrix_train[0, 1]:.4f}")
print(f"  - Correlation coefficient: {cov_matrix_train[0, 1] / np.sqrt(cov_matrix_train[0, 0] * cov_matrix_train[1, 1]):.6f}")

# Test set covariance
cov_matrix_test = np.cov(y_test, y_pred_test)
print(f"\n📊 Test Set Covariance Matrix:")
print(f"  - Var(Actual): {cov_matrix_test[0, 0]:.4f}")
print(f"  - Var(Predicted): {cov_matrix_test[1, 1]:.4f}")
print(f"  - Cov(Actual, Predicted): {cov_matrix_test[0, 1]:.4f}")
print(f"  - Correlation coefficient: {cov_matrix_test[0, 1] / np.sqrt(cov_matrix_test[0, 0] * cov_matrix_test[1, 1]):.6f}")

# Pearson and Spearman correlations
from scipy.stats import pearsonr, spearmanr

pearson_train, p_pearson_train = pearsonr(y_train, y_pred_train)
spearman_train, p_spearman_train = spearmanr(y_train, y_pred_train)

pearson_test, p_pearson_test = pearsonr(y_test, y_pred_test)
spearman_test, p_spearman_test = spearmanr(y_test, y_pred_test)

print(f"\n📊 Correlation Analysis (Training):")
print(f"  - Pearson correlation: {pearson_train:.6f} (p-value: {p_pearson_train:.2e})")
print(f"  - Spearman correlation: {spearman_train:.6f} (p-value: {p_spearman_train:.2e})")

print(f"\n📊 Correlation Analysis (Test):")
print(f"  - Pearson correlation: {pearson_test:.6f} (p-value: {p_pearson_test:.2e})")
print(f"  - Spearman correlation: {spearman_test:.6f} (p-value: {p_spearman_test:.2e})")

# Interpretation
print(f"\n💡 Interpretation:")
print(f"  - High Pearson r indicates strong linear relationship")
print(f"  - High Spearman ρ indicates strong monotonic relationship")
print(f"  - p-values < 0.05 indicate statistical significance")
if pearson_test > 0.95 and p_pearson_test < 0.001:
    print(f"  ✓ Excellent correlation between predictions and actual values")

### 8.3 Feature Distribution Analysis

In [ ]:
# Comprehensive feature statistics
print("="*70)
print("FEATURE DISTRIBUTION STATISTICS")
print("="*70)

# Calculate statistics for all features
feature_stats = pd.DataFrame({
    'Feature': X.columns,
    'Mean': X.mean(),
    'Std': X.std(),
    'Min': X.min(),
    'Q25': X.quantile(0.25),
    'Median': X.median(),
    'Q75': X.quantile(0.75),
    'Max': X.max(),
    'Variance': X.var(),
    'Skewness': X.skew(),
    'Kurtosis': X.kurtosis()
})

# Sort by variance (descending)
feature_stats_sorted = feature_stats.sort_values('Variance', ascending=False)

print(f"\n📊 Top 10 Features by Variance:")
print(feature_stats_sorted[['Feature', 'Mean', 'Std', 'Variance', 'Min', 'Max']].head(10).to_string(index=False))

print(f"\n📊 Overall Feature Statistics Summary:")
print(f"  - Mean of means: {feature_stats['Mean'].mean():.4f}")
print(f"  - Mean of standard deviations: {feature_stats['Std'].mean():.4f}")
print(f"  - Mean of variances: {feature_stats['Variance'].mean():.4f}")
print(f"  - Total features: {len(feature_stats)}")

# Identify features with extreme skewness or kurtosis
extreme_skew = feature_stats[np.abs(feature_stats['Skewness']) > 2]
extreme_kurt = feature_stats[np.abs(feature_stats['Kurtosis']) > 5]

print(f"\n⚠️  Features with Extreme Skewness (|skew| > 2): {len(extreme_skew)}")
if len(extreme_skew) > 0:
    print(extreme_skew[['Feature', 'Skewness']].to_string(index=False))

print(f"\n⚠️  Features with Extreme Kurtosis (|kurt| > 5): {len(extreme_kurt)}")
if len(extreme_kurt) > 0:
    print(extreme_kurt[['Feature', 'Kurtosis']].head(10).to_string(index=False))

# Save detailed statistics to CSV
stats_path = reports_dir / 'feature_statistics.csv'
feature_stats_sorted.to_csv(stats_path, index=False)
print(f"\n✓ Full feature statistics saved to: {stats_path}")

### 8.4 Feature Importance: Summary Statistics

In [ ]:
# Extract feature importance from Random Forest
feature_importance = best_rf.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

print("="*70)
print("FEATURE IMPORTANCE SUMMARY STATISTICS")
print("="*70)

print(f"\n📊 Importance Distribution:")
print(f"  - Minimum: {feature_importance.min():.6f}")
print(f"  - Maximum: {feature_importance.max():.6f}")
print(f"  - Mean: {feature_importance.mean():.6f}")
print(f"  - Median: {np.median(feature_importance):.6f}")
print(f"  - Std Dev: {feature_importance.std():.6f}")
print(f"  - Variance: {feature_importance.var():.6f}")
print(f"  - Sum: {feature_importance.sum():.6f} (should be ≈1.0)")

print(f"\n📊 Percentiles:")
for p in [10, 25, 50, 75, 90, 95, 99]:
    print(f"  - {p}th percentile: {np.percentile(feature_importance, p):.6f}")

print(f"\n📊 Top 15 Most Important Features:")
top_features = feature_importance_df.head(15)
for idx, row in top_features.iterrows():
    print(f"  {row['Feature']:30s} : {row['Importance']:.6f} ({100*row['Importance']:.2f}%)")

# Calculate cumulative importance
feature_importance_df['Cumulative_Importance'] = feature_importance_df['Importance'].cumsum()
n_features_80 = (feature_importance_df['Cumulative_Importance'] <= 0.80).sum() + 1
n_features_90 = (feature_importance_df['Cumulative_Importance'] <= 0.90).sum() + 1
n_features_95 = (feature_importance_df['Cumulative_Importance'] <= 0.95).sum() + 1

print(f"\n📊 Cumulative Importance Analysis:")
print(f"  - Features for 80% importance: {n_features_80} / {len(feature_names)} ({100*n_features_80/len(feature_names):.1f}%)")
print(f"  - Features for 90% importance: {n_features_90} / {len(feature_names)} ({100*n_features_90/len(feature_names):.1f}%)")
print(f"  - Features for 95% importance: {n_features_95} / {len(feature_names)} ({100*n_features_95/len(feature_names):.1f}%)")

# Identify dominant features
dominant_features = feature_importance_df[feature_importance_df['Importance'] > 0.05]
print(f"\n📊 Dominant Features (importance > 5%): {len(dominant_features)}")
for idx, row in dominant_features.iterrows():
    print(f"  {row['Feature']:30s} : {100*row['Importance']:.2f}%")

# Save feature importance
importance_path = reports_dir / 'feature_importance_rf.csv'
feature_importance_df.to_csv(importance_path, index=False)
print(f"\n✓ Feature importance saved to: {importance_path}")

In [ ]:
# Visualize feature importance distribution
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Top 20 features bar plot
ax1 = axes[0, 0]
top_20 = feature_importance_df.head(20)
ax1.barh(range(len(top_20)), top_20['Importance'], color='steelblue', edgecolor='black')
ax1.set_yticks(range(len(top_20)))
ax1.set_yticklabels(top_20['Feature'], fontsize=9)
ax1.set_xlabel('Importance', fontsize=11, fontweight='bold')
ax1.set_title('Top 20 Most Important Features', fontsize=12, fontweight='bold')
ax1.invert_yaxis()
ax1.grid(True, alpha=0.3, axis='x')

# Importance distribution histogram
ax2 = axes[0, 1]
ax2.hist(feature_importance, bins=50, color='darkgreen', alpha=0.7, edgecolor='black')
ax2.axvline(feature_importance.mean(), color='red', linestyle='--', linewidth=2, 
            label=f'Mean: {feature_importance.mean():.5f}')
ax2.axvline(np.median(feature_importance), color='orange', linestyle='--', linewidth=2,
            label=f'Median: {np.median(feature_importance):.5f}')
ax2.set_xlabel('Feature Importance', fontsize=11, fontweight='bold')
ax2.set_ylabel('Frequency', fontsize=11, fontweight='bold')
ax2.set_title('Distribution of Feature Importance Values', fontsize=12, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

# Cumulative importance plot
ax3 = axes[1, 0]
sorted_importance = np.sort(feature_importance)[::-1]
cumsum_importance = np.cumsum(sorted_importance)
ax3.plot(range(1, len(cumsum_importance) + 1), cumsum_importance, linewidth=2, color='purple')
ax3.axhline(y=0.80, color='red', linestyle='--', linewidth=1.5, label='80% threshold')
ax3.axhline(y=0.90, color='orange', linestyle='--', linewidth=1.5, label='90% threshold')
ax3.axhline(y=0.95, color='green', linestyle='--', linewidth=1.5, label='95% threshold')
ax3.axvline(x=n_features_80, color='red', linestyle=':', linewidth=1.5, alpha=0.5)
ax3.axvline(x=n_features_90, color='orange', linestyle=':', linewidth=1.5, alpha=0.5)
ax3.set_xlabel('Number of Features', fontsize=11, fontweight='bold')
ax3.set_ylabel('Cumulative Importance', fontsize=11, fontweight='bold')
ax3.set_title(f'Cumulative Feature Importance\n{n_features_80} features = 80%, {n_features_90} features = 90%', 
              fontsize=12, fontweight='bold')
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3)

# Box plot of importance values
ax4 = axes[1, 1]
bp = ax4.boxplot([feature_importance], vert=False, patch_artist=True, widths=0.5,
                  boxprops=dict(facecolor='lightblue', edgecolor='black', linewidth=1.5),
                  medianprops=dict(color='red', linewidth=2),
                  whiskerprops=dict(color='black', linewidth=1.5),
                  capprops=dict(color='black', linewidth=1.5))
ax4.set_xlabel('Feature Importance', fontsize=11, fontweight='bold')
ax4.set_title(f'Feature Importance Distribution Summary\nMin={feature_importance.min():.5f}, Max={feature_importance.max():.5f}', 
              fontsize=12, fontweight='bold')
ax4.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(figures_dir / 'feature_importance_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Figure saved: {figures_dir / 'feature_importance_analysis.png'}")

### 8.5 Comprehensive Statistical Summary Report

In [ ]:
# Generate comprehensive statistical report
statistical_summary = {
    'Model': 'Random Forest',
    'Experiment_Date': datetime.now().strftime('%Y-%m-%d'),
    
    # Model Performance
    'Train_R2': train_r2,
    'Test_R2': test_r2,
    'Train_RMSE': train_rmse,
    'Test_RMSE': test_rmse,
    'Train_MAE': train_mae,
    'Test_MAE': test_mae,
    'Overfitting_Gap': overfit_gap,
    
    # Residual Statistics - Training
    'Train_Residual_Mean': residuals_train.mean(),
    'Train_Residual_Std': residuals_train.std(),
    'Train_Residual_Variance': residuals_train.var(),
    'Train_Residual_Skewness': pd.Series(residuals_train).skew(),
    'Train_Residual_Kurtosis': pd.Series(residuals_train).kurtosis(),
    'Train_Residual_Min': residuals_train.min(),
    'Train_Residual_Max': residuals_train.max(),
    
    # Residual Statistics - Test
    'Test_Residual_Mean': residuals_test.mean(),
    'Test_Residual_Std': residuals_test.std(),
    'Test_Residual_Variance': residuals_test.var(),
    'Test_Residual_Skewness': pd.Series(residuals_test).skew(),
    'Test_Residual_Kurtosis': pd.Series(residuals_test).kurtosis(),
    'Test_Residual_Min': residuals_test.min(),
    'Test_Residual_Max': residuals_test.max(),
    
    # Covariance
    'Train_Var_Actual': cov_matrix_train[0, 0],
    'Train_Var_Predicted': cov_matrix_train[1, 1],
    'Train_Covariance': cov_matrix_train[0, 1],
    'Test_Var_Actual': cov_matrix_test[0, 0],
    'Test_Var_Predicted': cov_matrix_test[1, 1],
    'Test_Covariance': cov_matrix_test[0, 1],
    
    # Correlations
    'Train_Pearson_r': pearson_train,
    'Train_Pearson_p': p_pearson_train,
    'Train_Spearman_rho': spearman_train,
    'Train_Spearman_p': p_spearman_train,
    'Test_Pearson_r': pearson_test,
    'Test_Pearson_p': p_pearson_test,
    'Test_Spearman_rho': spearman_test,
    'Test_Spearman_p': p_spearman_test,
    
    # Feature Importance
    'Feature_Importance_Min': feature_importance.min(),
    'Feature_Importance_Max': feature_importance.max(),
    'Feature_Importance_Mean': feature_importance.mean(),
    'Feature_Importance_Median': np.median(feature_importance),
    'Feature_Importance_Std': feature_importance.std(),
    'Feature_Importance_Variance': feature_importance.var(),
    'N_Features_80pct': n_features_80,
    'N_Features_90pct': n_features_90,
    'N_Features_95pct': n_features_95,
    'N_Dominant_Features': len(dominant_features),
    
    # Model Configuration
    'N_Estimators': best_rf.n_estimators,
    'Max_Depth': best_rf.max_depth,
    'Min_Samples_Split': best_rf.min_samples_split,
    'Min_Samples_Leaf': best_rf.min_samples_leaf,
    'Max_Features': best_rf.max_features if isinstance(best_rf.max_features, float) else str(best_rf.max_features),
    
    # Dataset Info
    'N_Train_Samples': len(y_train),
    'N_Test_Samples': len(y_test),
    'N_Features': len(feature_names)
}

# Convert to DataFrame for nice display
summary_df = pd.DataFrame([statistical_summary]).T
summary_df.columns = ['Value']

# Save to CSV
summary_path = reports_dir / 'statistical_summary_rf.csv'
summary_df.to_csv(summary_path)

print("="*70)
print("COMPREHENSIVE STATISTICAL SUMMARY")
print("="*70)
print("\n📊 All key statistics have been calculated and saved.")
print(f"✓ Summary saved to: {summary_path}")
print(f"\n📋 Summary Preview (first 20 metrics):")
print(summary_df.head(20).to_string())

print(f"\n\n💡 PhD Professor Feedback Addressed:")
print(f"  ✓ Covariance matrices calculated for train and test sets")
print(f"  ✓ Residual variance and distribution analyzed")
print(f"  ✓ Mean squared error decomposed")
print(f"  ✓ Feature distributions summarized (min, max, mean, variance)")
print(f"  ✓ Feature importance statistics computed")
print(f"  ✓ Normality tests performed (Q-Q plots)")
print(f"  ✓ All metrics saved to CSV files for further analysis")

## 9. ONNX Model Export for Production

Convert the trained Random Forest to ONNX format for cross-platform deployment and efficient inference.

In [ ]:
# Export Random Forest to ONNX format
print("🔄 Converting Random Forest to ONNX format...")

# Define input type for ONNX conversion
n_features = X_train_processed.shape[1]
initial_type = [('float_input', FloatTensorType([None, n_features]))]

# Convert model
onnx_model = convert_sklearn(
    best_rf, 
    initial_types=initial_type,
    target_opset=12
)

# Save ONNX model
onnx_path = models_dir / 'random_forest_model.onnx'
with open(onnx_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"✓ ONNX model saved to: {onnx_path}")
print(f"✓ Model size: {onnx_path.stat().st_size / 1024:.2f} KB")

# Validate ONNX model by making predictions
print("\n🔄 Validating ONNX model...")
sess = rt.InferenceSession(str(onnx_path))
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

# Make predictions with ONNX
X_test_float = X_test_processed.astype(np.float32)
onnx_pred = sess.run([label_name], {input_name: X_test_float})[0]

# Compare ONNX predictions with sklearn predictions
prediction_diff = np.abs(y_pred_test - onnx_pred.flatten())
max_diff = prediction_diff.max()
mean_diff = prediction_diff.mean()

print(f"\n📊 ONNX Validation Results:")
print(f"  - Max prediction difference: {max_diff:.6f} years")
print(f"  - Mean prediction difference: {mean_diff:.6f} years")
print(f"  - Predictions match sklearn: {np.allclose(y_pred_test, onnx_pred.flatten(), rtol=1e-5)}")

if max_diff < 1e-4:
    print(f"  ✓ EXCELLENT: ONNX model predictions match sklearn perfectly")
elif max_diff < 1e-2:
    print(f"  ✓ GOOD: ONNX model predictions are very close to sklearn")
else:
    print(f"  ⚠️  WARNING: Significant differences detected")

# Save sklearn model as well
sklearn_model_path = models_dir / 'random_forest_model.pkl'
joblib.dump(best_rf, sklearn_model_path)
print(f"\n✓ Scikit-learn model saved to: {sklearn_model_path}")

## 10. SHAP Explanations for Model Interpretability

Use SHAP TreeExplainer to generate feature importance and individual prediction explanations.

In [ ]:
# Initialize SHAP TreeExplainer (optimized for tree-based models)
print("🔄 Initializing SHAP TreeExplainer...")
explainer = shap.TreeExplainer(best_rf)

# Calculate SHAP values for test set (use a sample for speed if dataset is large)
sample_size = min(500, len(X_test_processed))
X_test_sample = X_test_processed[:sample_size]

print(f"🔄 Calculating SHAP values for {sample_size} test samples...")
shap_values = explainer.shap_values(X_test_sample)

print(f"✓ SHAP values calculated")
print(f"✓ SHAP values shape: {shap_values.shape}")
print(f"✓ Base value (expected value): {explainer.expected_value:.4f} years")

In [ ]:
# SHAP Summary Plot (feature importance across all samples)
print("📊 Generating SHAP summary plot...")
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test_sample, feature_names=feature_names, show=False, max_display=20)
plt.title('SHAP Feature Importance Summary\n(Top 20 Features)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig(figures_dir / 'shap_summary_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Figure saved: {figures_dir / 'shap_summary_plot.png'}")

In [ ]:
# SHAP Bar Plot (mean absolute SHAP values)
print("📊 Generating SHAP bar plot...")
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test_sample, feature_names=feature_names, 
                  plot_type="bar", show=False, max_display=20)
plt.title('SHAP Feature Importance (Mean |SHAP value|)\nTop 20 Features', 
          fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig(figures_dir / 'shap_bar_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Figure saved: {figures_dir / 'shap_bar_plot.png'}")

## 11. MLflow Experiment Tracking

Log all metrics, parameters, and artifacts to MLflow for comparison with the Linear Regression baseline.

In [ ]:
# Set experiment name (same as Linear Regression for comparison)
EXPERIMENT_NAME = "aging_prediction_models"
mlflow.set_experiment(EXPERIMENT_NAME)

print(f"🔄 Logging to MLflow experiment: {EXPERIMENT_NAME}")

with mlflow.start_run(run_name="random_forest_optimized"):
    
    # Log hyperparameters
    mlflow.log_params({
        'model': 'RandomForestRegressor',
        'n_estimators': best_rf.n_estimators,
        'max_depth': best_rf.max_depth,
        'min_samples_split': best_rf.min_samples_split,
        'min_samples_leaf': best_rf.min_samples_leaf,
        'max_features': best_rf.max_features if isinstance(best_rf.max_features, float) else str(best_rf.max_features),
        'bootstrap': best_rf.bootstrap,
        'random_state': RANDOM_STATE,
        'n_features': len(feature_names),
        'test_size': TEST_SIZE,
        'cv_folds': CV_FOLDS,
        'hp_search_iterations': N_ITER
    })
    
    # Log performance metrics
    mlflow.log_metrics({
        # Test set metrics
        'test_r2': test_r2,
        'test_rmse': test_rmse,
        'test_mae': test_mae,
        
        # Training set metrics
        'train_r2': train_r2,
        'train_rmse': train_rmse,
        'train_mae': train_mae,
        
        # Cross-validation metric
        'cv_mae': -random_search.best_score_,
        
        # Overfitting indicator
        'overfitting_gap': overfit_gap,
        
        # Residual statistics
        'test_residual_mean': residuals_test.mean(),
        'test_residual_std': residuals_test.std(),
        'test_residual_variance': residuals_test.var(),
        
        # Correlations
        'test_pearson_r': pearson_test,
        'test_spearman_rho': spearman_test,
        
        # Feature importance
        'feature_importance_max': feature_importance.max(),
        'feature_importance_mean': feature_importance.mean(),
        'n_features_80pct': n_features_80,
        'n_features_90pct': n_features_90
    })
    
    # Log models
    mlflow.sklearn.log_model(best_rf, "random_forest_model")
    
    # Log artifacts
    mlflow.log_artifact(str(sklearn_model_path))
    mlflow.log_artifact(str(onnx_path))
    mlflow.log_artifact(str(preprocessor_path))
    mlflow.log_artifact(str(importance_path))
    mlflow.log_artifact(str(summary_path))
    
    # Log figures
    for fig_file in figures_dir.glob('*.png'):
        mlflow.log_artifact(str(fig_file))
    
    # Log tags for easy filtering
    mlflow.set_tags({
        'model_type': 'ensemble',
        'model_family': 'tree_based',
        'optimization': 'randomized_search',
        'onnx_compatible': 'true',
        'shap_explanations': 'true',
        'issue': '#6',
        'experiment_date': datetime.now().strftime('%Y-%m-%d')
    })
    
    print("✓ Metrics logged to MLflow")
    print("✓ Parameters logged to MLflow")
    print("✓ Model artifacts logged to MLflow")
    print("✓ Figures logged to MLflow")
    print("✓ Tags set for easy filtering")
    
print(f"\n🎯 MLflow Run Complete!")
print(f"View results: mlflow ui")

## 12. Final Summary and Conclusions

This section provides a comprehensive summary of the Random Forest experiment, comparison with the Linear Regression baseline, and key insights.

In [ ]:
# ============================================================================
# COMPREHENSIVE EXPERIMENT SUMMARY
# ============================================================================

print("=" * 80)
print("RANDOM FOREST AGING PREDICTION MODEL - FINAL SUMMARY")
print("=" * 80)
print()

# Model Information
print("📊 MODEL ARCHITECTURE")
print("-" * 80)
print(f"  Algorithm:           RandomForestRegressor (Ensemble Method)")
print(f"  Optimization:        RandomizedSearchCV ({N_ITER} iterations, {CV_FOLDS}-fold CV)")
print(f"  Best Estimators:     {best_rf.n_estimators} trees")
print(f"  Max Depth:           {best_rf.max_depth}")
print(f"  Min Samples Split:   {best_rf.min_samples_split}")
print(f"  Min Samples Leaf:    {best_rf.min_samples_leaf}")
print(f"  Max Features:        {best_rf.max_features}")
print(f"  Bootstrap:           {best_rf.bootstrap}")
print()

# Dataset Information
print("📁 DATASET CHARACTERISTICS")
print("-" * 80)
print(f"  Total Samples:       {len(df)} (train: {len(X_train)}, test: {len(X_test)})")
print(f"  Total Features:      {len(feature_names)}")
print(f"  Target Variable:     Biological Age (years)")
print(f"  Age Range:           {y.min():.1f} - {y.max():.1f} years")
print(f"  Age Mean ± SD:       {y.mean():.1f} ± {y.std():.1f} years")
print(f"  Data Quality:        {df.isnull().sum().sum()} missing values")
print()

# Performance Metrics
print("🎯 MODEL PERFORMANCE")
print("-" * 80)
print(f"  Test R² Score:       {test_r2:.4f}  {'⭐' if test_r2 > 0.95 else '✓' if test_r2 > 0.90 else '⚠️'}")
print(f"  Test RMSE:           {test_rmse:.2f} years")
print(f"  Test MAE:            {test_mae:.2f} years")
print()
print(f"  Train R² Score:      {train_r2:.4f}")
print(f"  Train RMSE:          {train_rmse:.2f} years")
print(f"  Train MAE:           {train_mae:.2f} years")
print()
print(f"  CV MAE:              {-random_search.best_score_:.2f} years")
print(f"  Overfitting Gap:     {overfit_gap:.4f}  {'✓ Good' if overfit_gap < 0.03 else '⚠️ Moderate' if overfit_gap < 0.05 else '❌ High'}")
print()

# Statistical Analysis
print("📈 STATISTICAL ANALYSIS")
print("-" * 80)
print(f"  Residual Mean:       {residuals_test.mean():.4f} years (ideal: ~0)")
print(f"  Residual Std:        {residuals_test.std():.2f} years")
print(f"  Residual Variance:   {residuals_test.var():.2f}")
print(f"  Pearson r:           {pearson_test:.4f} (p={pearson_p_test:.2e})")
print(f"  Spearman ρ:          {spearman_test:.4f} (p={spearman_p_test:.2e})")
print()

# Feature Importance
print("🔍 FEATURE IMPORTANCE INSIGHTS")
print("-" * 80)
print(f"  Top Feature:         {top_features.iloc[0]['Feature']} ({top_features.iloc[0]['Importance']:.4f})")
print(f"  Top 5 Features:      {', '.join(top_features.head(5)['Feature'].values)}")
print(f"  Features (80%):      {n_features_80} features explain 80% of variance")
print(f"  Features (90%):      {n_features_90} features explain 90% of variance")
print(f"  Importance Range:    {feature_importance.min():.6f} - {feature_importance.max():.6f}")
print()

# Model Artifacts
print("💾 EXPORTED ARTIFACTS")
print("-" * 80)
print(f"  ✓ Random Forest Model:     {sklearn_model_path.name}")
print(f"  ✓ ONNX Model:              {onnx_path.name}")
print(f"  ✓ Preprocessor:            {preprocessor_path.name}")
print(f"  ✓ Feature Importance:      {importance_path.name}")
print(f"  ✓ Statistical Summary:     {summary_path.name}")
print(f"  ✓ Visualizations:          {len(list(figures_dir.glob('*.png')))} figures")
print()

# Baseline Comparison
print("📊 COMPARISON WITH LINEAR REGRESSION BASELINE (Issue #21)")
print("-" * 80)
baseline_r2 = 0.9691
baseline_mae = 2.05
print(f"  Linear Regression R²:      {baseline_r2:.4f}")
print(f"  Random Forest R²:          {test_r2:.4f}")
print(f"  R² Improvement:            {(test_r2 - baseline_r2):.4f} ({(test_r2 - baseline_r2)/baseline_r2*100:+.2f}%)")
print()
print(f"  Linear Regression MAE:     {baseline_mae:.2f} years")
print(f"  Random Forest MAE:         {test_mae:.2f} years")
print(f"  MAE Improvement:           {(baseline_mae - test_mae):.2f} years ({(baseline_mae - test_mae)/baseline_mae*100:+.2f}%)")
print()

# Interpretation
print("🎓 KEY FINDINGS")
print("-" * 80)
if test_r2 > baseline_r2:
    print("  ✓ Random Forest outperforms Linear Regression baseline")
    print("  ✓ Non-linear relationships in aging biomarkers captured effectively")
    print("  ✓ Ensemble approach reduces prediction variance")
else:
    print("  • Random Forest performs comparably to Linear Regression")
    print("  • Data may have predominantly linear relationships")
    print("  • Simpler model (Linear) might be preferred for interpretability")

if overfit_gap < 0.03:
    print("  ✓ Model generalizes well (minimal overfitting)")
elif overfit_gap < 0.05:
    print("  ⚠️ Moderate overfitting detected (consider regularization)")
else:
    print("  ❌ Significant overfitting (model too complex for data)")

if abs(residuals_test.mean()) < 0.5:
    print("  ✓ Predictions are unbiased (residual mean ≈ 0)")
else:
    print("  ⚠️ Systematic bias detected in predictions")

print()

# Limitations and Future Work
print("⚠️ LIMITATIONS")
print("-" * 80)
print("  • Model trained on simulated/synthetic data")
print("  • Feature interactions not explicitly modeled")
print("  • Cross-sectional data (longitudinal validation needed)")
print("  • Biological interpretability of ensemble models challenging")
print()

print("🚀 NEXT STEPS (From ROADMAP.md)")
print("-" * 80)
print("  1. Test model on independent validation cohort")
print("  2. Implement XGBoost (Issue #7) for gradient boosting comparison")
print("  3. Add biological pathway analysis with SHAP interactions")
print("  4. Deploy best model to production API")
print("  5. Conduct longitudinal validation study")
print()

print("=" * 80)
print(f"✓ Experiment completed successfully on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"✓ All results logged to MLflow experiment: {EXPERIMENT_NAME}")
print(f"✓ Artifacts saved to: {models_dir}")
print("=" * 80)